```python
# TODO: 找几个描述统计量卷可视化
# TODO: 实现Three-way ANOVA
# TODO: 实现多元独立性检验
# TODO: 多元逻辑斯蒂回归
```

In [66]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

In [67]:
df = pd.read_csv('cleaned_data.csv', index_col=0)
df.head()

,gender,major,grade,origin,money,1010,1011,1012,1013,0010,...,2102,2103,1110,1111,1112,1113,1000,1001,1002,1003
0,1,3,2,0,3000,0.435364,0.00000,0.564636,0.000000,0.000000,...,0.0,0.0,1.0,0.0,0.0,0.0,1.000000,0.000000,0.000000,0.0
1,1,1,2,2,4503,0.000000,0.00000,0.272892,0.727108,0.000000,...,0.0,0.0,0.0,1.0,0.0,0.0,0.000000,0.000000,0.000000,1.0
2,2,2,2,2,2134,0.000000,0.00000,1.000000,0.000000,0.000000,...,0.0,0.0,0.0,1.0,0.0,0.0,0.000000,0.000000,0.000000,1.0
3,2,2,2,1,2604,0.269976,0.32322,0.406804,0.000000,0.000000,...,0.0,0.0,0.0,1.0,0.0,0.0,0.000000,0.432907,0.567093,0.0
4,1,2,1,0,3000,0.399108,0.26753,0.333363,0.000000,0.189496,...,0.0,0.0,0.0,1.0,0.0,0.0,0.332225,0.367165,0.300610,0.0


In [68]:
def anova(df: pd.DataFrame, option: int):
    '''
    对某一个交通工具的softmax概率做anova
    '''
    zero_cols = df.columns[df.columns.str.endswith(str(option))]

    # 将所有0结尾的列转换为长格式
    zero_df = df[zero_cols].melt(var_name='category', value_name='prob')
    S = ['Distance', 'Weather', 'Constraint']
    for i, s in enumerate(S):
        zero_df[s] = zero_df['category'].str[i]

    zero_df.drop('category', axis=1, inplace=True)
    zero_df[S] = zero_df[S].astype(int)

    formula = f'prob ~ C({S[0]}) + C({S[1]}) + C({S[2]}) + C({S[0]}):C({S[1]}) + C({S[0]}):C({S[2]}) + C({S[1]}):C({S[2]}) + C({S[0]}):C({S[1]}):C({S[2]})'
    model = ols(formula, zero_df).fit()
    anova_res = anova_lm(model)

    return anova_res

In [69]:
anova(df, 0)

,df,sum_sq,mean_sq,F,PR(>F)
C(Distance),2.0,25.704493,12.852246,158.374513,1.260813e-60
C(Weather),1.0,0.267547,0.267547,3.296901,6.970422e-02
C(Constraint),1.0,2.948794,2.948794,36.337135,2.316561e-09
C(Distance):C(Weather),2.0,1.691500,0.845750,10.421931,3.307035e-05
C(Distance):C(Constraint),2.0,3.676023,1.838012,22.649284,2.375212e-10
C(Weather):C(Constraint),1.0,0.081415,0.081415,1.003256,3.167614e-01
C(Distance):C(Weather):C(Constraint),2.0,0.262162,0.131081,1.615275,1.993441e-01
Residual,1020.0,82.773997,0.081151,NaN,NaN


In [70]:
anova(df, 1)

,df,sum_sq,mean_sq,F,PR(>F)
C(Distance),2.0,4.767242,2.383621,25.541614,1.500535e-11
C(Weather),1.0,9.788587,9.788587,104.889297,1.693486e-23
C(Constraint),1.0,10.376253,10.376253,111.186411,9.574503e-25
C(Distance):C(Weather),2.0,4.817967,2.408984,25.813386,1.158442e-11
C(Distance):C(Constraint),2.0,1.849894,0.924947,9.911240,5.456252e-05
C(Weather):C(Constraint),1.0,3.878330,3.878330,41.558121,1.762262e-10
C(Distance):C(Weather):C(Constraint),2.0,5.323056,2.661528,28.519520,8.872590e-13
Residual,1020.0,95.189494,0.093323,NaN,NaN


In [71]:
anova(df, 2)

,df,sum_sq,mean_sq,F,PR(>F)
C(Distance),2.0,25.264952,12.632476,186.241284,1.130593e-69
C(Weather),1.0,6.935398,6.935398,102.248959,5.678035e-23
C(Constraint),1.0,0.268616,0.268616,3.960224,4.685411e-02
C(Distance):C(Weather),2.0,3.491657,1.745829,25.738849,1.243622e-11
C(Distance):C(Constraint),2.0,0.279014,0.139507,2.056757,1.283980e-01
C(Weather):C(Constraint),1.0,1.799275,1.799275,26.526811,3.118443e-07
C(Distance):C(Weather):C(Constraint),2.0,1.794274,0.897137,13.226539,2.133094e-06
Residual,1020.0,69.185118,0.067829,NaN,NaN


In [72]:
anova(df, 3)

,df,sum_sq,mean_sq,F,PR(>F)
C(Distance),2.0,4.726694,2.363347,53.395220,8.832411e-23
C(Weather),1.0,0.000488,0.000488,0.011027,9.163907e-01
C(Constraint),1.0,0.971667,0.971667,21.952929,3.172332e-06
C(Distance):C(Weather),2.0,0.138526,0.069263,1.564866,2.096177e-01
C(Distance):C(Constraint),2.0,0.727463,0.363732,8.217806,2.880719e-04
C(Weather):C(Constraint),1.0,0.117405,0.117405,2.652526,1.036943e-01
C(Distance):C(Weather):C(Constraint),2.0,0.266390,0.133195,3.009280,4.976532e-02
Residual,1020.0,45.146627,0.044261,NaN,NaN
